# Gesture Recognition
Project submitted by:
- Swapnil Disawal
- Raja Rajalakshmi.K 

A home electronics company that manufactures state of the art smart televisions want to develop a cool feature in the smart-TV that can recognize five different gestures performed by the user which will help users control the TV without using a remote.
The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:
- 1.Thumbs up:  Increase the volume
- 2.Thumbs down: Decrease the volume
- 3.Left swipe: 'Jump' backwards 10 seconds
- 4.Right swipe: 'Jump' forward 10 seconds  
- 5.Stop: Pause the movie

In [73]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [74]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. Keep increasing the batch size until the machine throws an error.

In [75]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size = 10 #experiment with the batch size

***Note*** in our implementaiton true **batch_size** is always 2 fold as we agument our data. if if batch size is 10, true batch size at the run time is 20, as twice the number of data set loaded are added to the batch


## Generator
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [76]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
#total_frames = 30
nb_frames = 30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

In [77]:
#  generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [78]:
#  normalise the data
def normalize_data(data):
    return data/127.5-1

In [79]:
#  normalise the data 2
def normalize_image(img):
    normalized_image= img - np.percentile(img,15)/ np.percentile(img,85) - np.percentile(img,15)
    return normalized_image

In [80]:
#checked the data and if we crop more than 10 pixels some of images having dimesions 
#120X160 are loosing the hand object in image. hence cropping  by 10X10 and then resizing so image wont shrink much too
def crop_and_resize_image(img):
    if img.shape[0] != img.shape[1]:
        img=img[0:120,10:150]
    resized_image = imresize(img, (nb_rows,nb_cols))
    return resized_image

In [81]:
# initialize all the batch image data and labels
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [82]:
# load batch data
def load_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    
    #  augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    
    #augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = init_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            ## image = imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping uneven frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them.    
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [83]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield load_batch_images(source_path, folder_list, batch, batch_size, t,validation)
            

        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield load_batch_images(source_path, folder_list, batch, batch_size, t,validation)


In [84]:
# for RNN
def get_batchdata_affinetransform(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    
    # We will also build an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    
   
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # resize image
            
            resized_image=crop_and_resize_image(image)
            
            #Normal data
            batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0])#normalise and feed in the image
            batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1])#normalise and feed in the image
            batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2])#normalise and feed in the image
            
           #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized_image, M, (resized_image.shape[0], resized_image.shape[1])))
            

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
               
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 

    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [85]:
# for RNN
def generator_augmentation(source_path, folder_list, batch_size, validation=False,ablation=None):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    if(ablation!=None):
        folder_list=folder_list[:ablation]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield get_batchdata_affinetransform(source_path, folder_list, batch, batch_size, t,validation)
            
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield get_batchdata_affinetransform(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [86]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Final Model

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

## CNN_Model1

In [16]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 184s 3s/step - loss: 1.7311 - categorical_accuracy: 0.2613 - val_loss: 1.2905 - val_categorical_accuracy: 0.4300

Epoch 00001: saving model to model_init_2020-07-1220_19_19.720565/model-00001-1.73360-0.25943-1.29050-0.43000.h5
Epoch 2/20
67/67 [==============================] - 40s 604ms/step - loss: 1.5027 - categorical_accuracy: 0.3300 - val_loss: 1.1888 - val_categorical_accuracy: 0.4800

Epoch 00002: saving model to model_init_2020-07-1220_19_19.720565/model-00002-1.50275-0.33002-1.18878-0.48000.h5
Epoch 3/20
67/67 [==============================] - 35s 526ms/step - loss: 1.3387 - categorical_accuracy: 0.4063 - val_loss: 1.1991 - val_categorical_accuracy: 0.5400

Epoch 00003: saving model to model_init_2020-07-1220_19_19.720565/model-00003-1.33868-0.40630-1.19913-0.54000.h5
Epoch 4/20
67/67 [==============================] - 28s 425ms/step - loss: 1.2946 -

## CNN_Model2

In [17]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 17s 499ms/step - loss: 0.3793 - categorical_accuracy: 0.8693 - val_loss: 0.5235 - val_categorical_accuracy: 0.7800

Epoch 00001: saving model to model_init_2020-07-1220_19_19.720565/model-00001-0.37929-0.86928-0.52352-0.78000.h5
Epoch 2/20
34/34 [==============================] - 14s 426ms/step - loss: 0.3257 - categorical_accuracy: 0.8725 - val_loss: 0.7019 - val_categorical_accuracy: 0.7200

Epoch 00002: saving model to model_init_2020-07-1220_19_19.720565/model-00002-0.32575-0.87255-0.70187-0.72000.h5
Epoch 3/20
34/34 [==============================] - 13s 393ms/step - loss: 0.3907 - categorical_accuracy: 0.8431 - val_loss: 0.3280 - val_categorical_accuracy: 0.8600

Epoch 00003: saving model to model_init_2020-07-1220_19_19.720565/model-00003-0.39073-0.84314-0.32804-0.86000.h5
Epoch 4/20
34/34 [==============================] - 14s 402ms/step - loss: 0.4846 - categorical_accuracy: 0.8268 - val_loss: 0.3478 - val_categorical_accurac

## CNN_Model3

In [18]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 10s 433ms/step - loss: 0.2231 - categorical_accuracy: 0.9179 - val_loss: 0.4628 - val_categorical_accuracy: 0.7500

Epoch 00001: saving model to model_init_2020-07-1220_19_19.720565/model-00001-0.22308-0.91787-0.46280-0.75000.h5
Epoch 2/30
23/23 [==============================] - 10s 428ms/step - loss: 0.2945 - categorical_accuracy: 0.8937 - val_loss: 0.5005 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2020-07-1220_19_19.720565/model-00002-0.29449-0.89372-0.50048-0.80000.h5
Epoch 3/30
23/23 [==============================] - 9s 388ms/step - loss: 0.2410 - categorical_accuracy: 0.9179 - val_loss: 0.2845 - val_categorical_accuracy: 0.9250

Epoch 00003: saving model to model_init_2020-07-1220_19_19.720565/model-00003-0.24102-0.91787-0.28454-0.92500.h5
Epoch 4/30
23/23 [==============================] - 10s 435ms/step - loss: 0.3017 - categorical_accuracy: 0.8889 - val_loss: 0.4062 - val_categorical_accuracy

## CNN_Model4

In [19]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 6s 463ms/step - loss: 0.1716 - categorical_accuracy: 0.9683 - val_loss: 0.3577 - val_categorical_accuracy: 0.9000

Epoch 00001: saving model to model_init_2020-07-1220_19_19.720565/model-00001-0.17155-0.96825-0.35766-0.90000.h5
Epoch 2/30
14/14 [==============================] - 6s 415ms/step - loss: 0.1511 - categorical_accuracy: 0.9524 - val_loss: 0.4889 - val_categorical_accuracy: 0.9000

Epoch 00002: saving model to model_init_2020-07-1220_19_19.720565/model-00002-0.15110-0.95238-0.48887-0.90000.h5
Epoch 3/30
14/14 [==============================] - 6s 447ms/step - loss: 0.2449 - categorical_accuracy: 0.9127 - val_loss: 0.2978 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-07-1220_19_19.720565/model-00003-0.24493-0.91270-0.29776-0.80000.h5
Epoch 4/30
14/14 [==============================] - 5s 383ms/step - loss: 0.4464 - categorical_accuracy: 0.7778 - val_loss: 0.2724 - val_categorical_accuracy: 0

## CNN_Model5

In [20]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 4s 512ms/step - loss: 0.0957 - categorical_accuracy: 0.9583 - val_loss: 0.4288 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2020-07-1220_19_19.720565/model-00001-0.09574-0.95833-0.42879-0.80000.h5
Epoch 2/30
8/8 [==============================] - 4s 449ms/step - loss: 0.2991 - categorical_accuracy: 0.9028 - val_loss: 0.5371 - val_categorical_accuracy: 0.9000

Epoch 00002: saving model to model_init_2020-07-1220_19_19.720565/model-00002-0.29905-0.90278-0.53714-0.90000.h5
Epoch 3/30
8/8 [==============================] - 3s 432ms/step - loss: 0.2238 - categorical_accuracy: 0.9306 - val_loss: 0.4172 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-07-1220_19_19.720565/model-00003-0.22383-0.93056-0.41720-0.80000.h5
Epoch 4/30
8/8 [==============================] - 3s 424ms/step - loss: 0.2864 - categorical_accuracy: 0.8750 - val_loss: 0.2190 - val_categorical_accuracy: 1.0000

E


Epoch 00027: saving model to model_init_2020-07-1220_19_19.720565/model-00027-0.15668-0.94444-0.27798-0.80000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 5.684342156072553e-17.
Epoch 28/30
8/8 [==============================] - 3s 427ms/step - loss: 0.3612 - categorical_accuracy: 0.8472 - val_loss: 0.6617 - val_categorical_accuracy: 0.8500

Epoch 00028: saving model to model_init_2020-07-1220_19_19.720565/model-00028-0.36122-0.84722-0.66175-0.85000.h5
Epoch 29/30
8/8 [==============================] - 4s 444ms/step - loss: 0.3135 - categorical_accuracy: 0.8333 - val_loss: 0.2980 - val_categorical_accuracy: 0.9000

Epoch 00029: saving model to model_init_2020-07-1220_19_19.720565/model-00029-0.31352-0.83333-0.29799-0.90000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 2.842171078036277e-17.
Epoch 30/30
8/8 [==============================] - 4s 453ms/step - loss: 0.1628 - categorical_accuracy: 0.9444 - val_loss: 0.4444 - val_categorical_accuracy: 0.7500



## CNN_Model6

In [24]:
batch_size = 8
num_epochs = 25

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/25
83/83 [==============================] - 37s 443ms/step - loss: 0.3055 - categorical_accuracy: 0.8942 - val_loss: 0.4189 - val_categorical_accuracy: 0.8538

Epoch 00001: saving model to model_init_2020-07-1220_19_19.720565/model-00001-0.30549-0.89424-0.41889-0.85385.h5
Epoch 2/25
83/83 [==============================] - 34s 413ms/step - loss: 0.2465 - categorical_accuracy: 0.9143 - val_loss: 0.3479 - val_categorical_accuracy: 0.8462

Epoch 00002: saving model to model_init_2020-07-1220_19_19.720565/model-00002-0.24647-0.91432-0.34788-0.84615.h5
Epoch 3/25
83/83 [==============================] - 33s 398ms/step - loss: 0.3053 - categorical_accuracy: 0.8728 - val_loss: 0.4014 - val_categorical_accuracy: 0.8385

Epoch 00003: saving model to model_init_2020-07-1220_19_19.720565/model-00003-0.30527-0.87282-0.40143-0.83846.h5
Epoch 4/25
83/83 [==============================] - 36s 437ms/step - loss: 0.2668 - categorical_accuracy: 0.9063 - val_loss: 0.4522 - val_categorical_accurac

## RNN_Model

In [87]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D,Conv2D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers.recurrent import LSTM, GRU

#write your model here

#RNN
# input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)
nb_classes = 5
nb_featuremap = [8,16,32,64,128,256]
nb_dense = [128,64,5]

model = Sequential()



model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))

model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))

model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))

model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))


model.add(GRU(128, return_sequences=False))
model.add(Dense(nb_classes, activation='softmax'))

In [88]:
from keras.optimizers import Adam

optimiser = Adam()#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_37 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_38 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_39 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_40 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_41 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_42 (TimeDis (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_43 (TimeDis (None, 30, 7, 7, 64)      0         
__________

In [89]:
train_generator = generator_augmentation(train_path, train_doc, batch_size)

val_generator = generator_augmentation(train_path, train_doc, batch_size)

In [90]:
model_name = 'model_rnn/model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [91]:
batch_size = 25
num_epochs = 16


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [92]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  train ; batch size = 10
Source path =  train ; batch size =Epoch 1/16
 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


27/27 [==============================] - 56s 2s/step - loss: 1.3449 - categorical_accuracy: 0.4296 - val_loss: 1.3229 - val_categorical_accuracy: 0.4750

Epoch 00001: saving model to model_rnn/model_init_2020-07-1221_14_58.428971/model-00001-1.34491-0.42963-1.32294-0.47500.h5
Epoch 2/16
27/27 [==============================] - 46s 2s/step - loss: 1.2433 - categorical_accuracy: 0.4704 - val_loss: 0.8760 - val_categorical_accuracy: 0.6375

Epoch 00002: saving model to model_rnn/model_init_2020-07-1221_14_58.428971/model-00002-1.24325-0.47037-0.87599-0.63750.h5
Epoch 3/16
27/27 [==============================] - 31s 1s/step - loss: 1.0094 - categorical_accuracy: 0.5932 - val_loss: 1.5392 - val_categorical_accuracy: 0.5375

Epoch 00003: saving model to model_rnn/model_init_2020-07-1221_14_58.428971/model-00003-1.00313-0.57576-1.53918-0.53750.h5
Epoch 4/16
27/27 [==============================] - 17s 644ms/step - loss: 1.3239 - categorical_accuracy: 0.5185 - val_loss: 1.1664 - val_categoric

### Summary of the metrics for the final model

| Model | Categorical-Accuracy| Validation_categorical _Accuracy|
| --- | --- | --- |
| CNN_Model 1 |0.81 | 78 |
| CNN_Model 2 |0.91| 0.90 |
| CNN_Model 3 |0.89| 0.95|
| CNN_Model 4 |0.91| 0.90|
| CNN_Model 5 |0.89 | 0.85|
| CNN_Model 6 |0.88 | 0.90 |
| RNN_model |0.89 | 0.97 |


### Conclusion:
We chose CNN_ model 4 with batch size :50 and epoch :30

Reasons:
- time consumed by the model for training was between 5s-6s per epoch which much better the other models used in the experiment
- Since the model used will used on daily basis for watching the television the response rate needs to high so we decided to go with categorical accuracy of 91 and validation accuracy of 90
